In [21]:
import matplotlib
from datascience import Table
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')

from datascience.predicates import are
import subprocess
import os

In [24]:
def total_cost(cost_table):
    return sum(cost_table.apply(lambda row: row[4]))

In [7]:
costs_1 = Table.read_table('costs/outputs-dijkstra-degree-greedy.csv')
costs_2 = Table.read_table('costs/outputs.csv')

In [25]:
print(total_cost(costs_1))
print(total_cost(costs_2))

2445618540423.908
2832819276532.0205


In [8]:
costs_1

output_dir,output_filename,travel_cost,conquer_cost,cost
outputs-dijkstra-degree-greedy,0.out,28,40,68
outputs-dijkstra-degree-greedy,1.out,194,52,246
outputs-dijkstra-degree-greedy,10.out,250.8,281.31,532.11
outputs-dijkstra-degree-greedy,100.out,403864,27704,431568
outputs-dijkstra-degree-greedy,101.out,324,1,325
outputs-dijkstra-degree-greedy,105.out,1.78909e+07,1.80057e+07,3.58966e+07
outputs-dijkstra-degree-greedy,106.out,3.59132e+07,3.52362e+07,7.11494e+07
outputs-dijkstra-degree-greedy,107.out,9.28911e+06,6.61577e+06,1.59049e+07
outputs-dijkstra-degree-greedy,108.out,10380,2278.12,12658.1
outputs-dijkstra-degree-greedy,109.out,9850.71,1758.54,11609.3


In [9]:
costs_2

output_dir,output_filename,travel_cost,conquer_cost,cost
outputs,0.out,16,51,67
outputs,1.out,194,52,246
outputs,10.out,305.62,351.85,657.47
outputs,100.out,403864,27704,431568
outputs,101.out,324,1,325
outputs,105.out,2.41584e+07,1.48975e+07,3.90559e+07
outputs,106.out,4.46618e+07,2.97175e+07,7.43793e+07
outputs,107.out,1.23333e+07,6.44751e+06,1.87808e+07
outputs,108.out,13308.1,3376.05,16684.2
outputs,109.out,10799.8,1880.66,12680.5


In [10]:
d = costs_1.join('output_filename', costs_2)
d.where("cost", are.above, "cost_2")
# d

output_filename,output_dir,travel_cost,conquer_cost,cost,output_dir_2,travel_cost_2,conquer_cost_2,cost_2
0.out,outputs-dijkstra-degree-greedy,28,40,68,outputs,16,51,67
117.out,outputs-dijkstra-degree-greedy,375,602,977,outputs,376,520,896
118.out,outputs-dijkstra-degree-greedy,775,1206,1981,outputs,776,1040,1816
119.out,outputs-dijkstra-degree-greedy,176,300,476,outputs,176,260,436
121.out,outputs-dijkstra-degree-greedy,63248,1845,65093,outputs,63300,1171,64471
122.out,outputs-dijkstra-degree-greedy,786,516,1302,outputs,810,452,1262
126.out,outputs-dijkstra-degree-greedy,274,720,994,outputs,306,600,906
145.out,outputs-dijkstra-degree-greedy,992.26,1048,2040.26,outputs,1141.12,837,1978.12
146.out,outputs-dijkstra-degree-greedy,133,91,224,outputs,149,65,214
157.out,outputs-dijkstra-degree-greedy,32,189064,189096,outputs,30,173498,173528


In [11]:
appended = costs_1.copy().append(costs_2).sort('output_filename')
appended

output_dir,output_filename,travel_cost,conquer_cost,cost
outputs-dijkstra-degree-greedy,0.out,28,40,68
outputs,0.out,16,51,67
outputs-dijkstra-degree-greedy,1.out,194,52,246
outputs,1.out,194,52,246
outputs-dijkstra-degree-greedy,10.out,250.8,281.31,532.11
outputs,10.out,305.62,351.85,657.47
outputs-dijkstra-degree-greedy,100.out,403864,27704,431568
outputs,100.out,403864,27704,431568
outputs-dijkstra-degree-greedy,101.out,324,1,325
outputs,101.out,324,1,325


In [12]:
with_min = appended.group('output_filename', min).select(['output_filename', 'cost min'])
with_min

output_filename,cost min
0.out,67
1.out,246
10.out,532.11
100.out,431568
101.out,325
105.out,3.58966e+07
106.out,7.11494e+07
107.out,1.59049e+07
108.out,12658.1
109.out,11609.3


In [28]:
min_output_selected = appended.join('output_filename', with_min).where('cost', are.equal_to, 'cost min').drop('cost min')
min_output_selected.move_to_start('output_dir')
min_output_selected

output_dir,output_filename,travel_cost,conquer_cost,cost
outputs,0.out,16,51,67
outputs-dijkstra-degree-greedy,1.out,194,52,246
outputs,1.out,194,52,246
outputs-dijkstra-degree-greedy,10.out,250.8,281.31,532.11
outputs-dijkstra-degree-greedy,100.out,403864,27704,431568
outputs,100.out,403864,27704,431568
outputs-dijkstra-degree-greedy,101.out,324,1,325
outputs,101.out,324,1,325
outputs-dijkstra-degree-greedy,105.out,1.78909e+07,1.80057e+07,3.58966e+07
outputs-dijkstra-degree-greedy,106.out,3.59132e+07,3.52362e+07,7.11494e+07


In [23]:
# Remove duplicates
rows_to_take = []
seen = set()
for i in range(min_output_selected.num_rows):
    filename = min_output_selected.row(i)[1]
    if filename not in seen:
        seen.add(filename)
        rows_to_take.append(i)
min_output_selected = min_output_selected.take(rows_to_take)
min_output_selected

output_dir,output_filename,travel_cost,conquer_cost,cost
outputs,0.out,16,51,67
outputs-dijkstra-degree-greedy,1.out,194,52,246
outputs-dijkstra-degree-greedy,10.out,250.8,281.31,532.11
outputs-dijkstra-degree-greedy,100.out,403864,27704,431568
outputs-dijkstra-degree-greedy,101.out,324,1,325
outputs-dijkstra-degree-greedy,105.out,1.78909e+07,1.80057e+07,3.58966e+07
outputs-dijkstra-degree-greedy,106.out,3.59132e+07,3.52362e+07,7.11494e+07
outputs-dijkstra-degree-greedy,107.out,9.28911e+06,6.61577e+06,1.59049e+07
outputs-dijkstra-degree-greedy,108.out,10380,2278.12,12658.1
outputs-dijkstra-degree-greedy,109.out,9850.71,1758.54,11609.3


In [26]:
# To implement: select files from min_out_selected and copy into a new folder
new_outputs_name = "outputs2"

if not os.path.exists(new_outputs_name):
    os.makedirs(new_outputs_name)

for i in range(min_output_selected.num_rows):
    row = min_output_selected.row(i)
    dirname = row[0]
    filename = row[1]
    subprocess.run(['cp', f'{dirname}/{filename}', f'{new_outputs_name}/{filename}'])

print(total_cost(min_output_selected))
# min_output_selected.to_csv(f'costs/{new_outputs_name}.csv')

2397365144778.0933
